In [1]:
pip install datasets pandas numpy matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install krippendorff


Note: you may need to restart the kernel to use updated packages.


In [3]:
import krippendorff
from datasets import load_dataset
import pandas as pd
import numpy as np


In [4]:
dataset = load_dataset("ucberkeley-dlab/measuring-hate-speech")


In [5]:
dataset["train"][0]


{'comment_id': 47777,
 'annotator_id': 10873,
 'platform': 3,
 'sentiment': 0.0,
 'respect': 0.0,
 'insult': 0.0,
 'humiliate': 0.0,
 'status': 2.0,
 'dehumanize': 0.0,
 'violence': 0.0,
 'genocide': 0.0,
 'attack_defend': 0.0,
 'hatespeech': 0.0,
 'hate_speech_score': -3.9,
 'text': 'Yes indeed. She sort of reminds me of the elder lady that played the part in the movie "Titanic" who was telling her story!!! And I wouldn\'t have wanted to cover who I really am!! I would be proud!!!! WE should be proud of our race no matter what it is!!',
 'infitms': 0.81,
 'outfitms': 1.88,
 'annotator_severity': 0.36,
 'std_err': 0.34,
 'annotator_infitms': 1.35,
 'annotator_outfitms': 1.23,
 'hypothesis': -1.1301777576839678,
 'target_race_asian': True,
 'target_race_black': True,
 'target_race_latinx': True,
 'target_race_middle_eastern': True,
 'target_race_native_american': True,
 'target_race_pacific_islander': True,
 'target_race_white': True,
 'target_race_other': False,
 'target_race': True,
 

In [6]:
df = pd.DataFrame(dataset["train"])

In [7]:
religion_cols = [c for c in df.columns if c.startswith("annotator_religion_")]
religion_cols


['annotator_religion_atheist',
 'annotator_religion_buddhist',
 'annotator_religion_christian',
 'annotator_religion_hindu',
 'annotator_religion_jewish',
 'annotator_religion_mormon',
 'annotator_religion_muslim',
 'annotator_religion_nothing',
 'annotator_religion_other']

In [8]:
df[religion_cols].sum().sort_values(ascending=False)


annotator_religion_christian    58573
annotator_religion_nothing      36790
annotator_religion_atheist      27143
annotator_religion_other         8511
annotator_religion_jewish        2171
annotator_religion_buddhist      2138
annotator_religion_mormon        1066
annotator_religion_muslim         969
annotator_religion_hindu          660
dtype: int64

In [32]:
# def score_label(score):
#     if score > 0.5:
#         return "Hate speech"
#     elif score < -1:
#         return "Counter"
#     else:
#         return "Neutral"

# df["score_label"] = df["hate_speech_score"].apply(score_label)




In [18]:
df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")

df["is_hate_speech_score"] = df["hate_speech_score"] > 0.5

religion_cols = [
    'annotator_religion_atheist',
    'annotator_religion_buddhist',
    'annotator_religion_christian',
    'annotator_religion_hindu',
    'annotator_religion_jewish',
    'annotator_religion_mormon',
    'annotator_religion_muslim',
    'annotator_religion_nothing',
    'annotator_religion_other'
]

rows = []
for rel in religion_cols:
    rel_name = rel.replace("annotator_religion_", "")
    subset = df[df[rel] == 1]

    total_n = len(subset)
    hate_n = subset["is_hate_speech_score"].sum()
    hate_pct = (hate_n / total_n * 100) if total_n > 0 else 0.0

    rows.append({
        "religion": rel_name,
        "n_annotations": total_n,
        "hate_speech_%": round(hate_pct, 2)
    })

hate_speech_by_religion = pd.DataFrame(rows).sort_values("n_annotations", ascending=False)
hate_speech_by_religion


,religion,n_annotations,hate_speech_%
1,christian,58573,36.04
2,nothing,36790,36.11
0,atheist,27143,36.78
3,other,8511,36.40


In [10]:

df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")

df["score_label"] = np.select(
    [
        df["hate_speech_score"] > 0.5,
        df["hate_speech_score"] < -1
    ],
    [
        "Hate Speech",
        "Counter "
    ],
    default="Neutral "
)

df["score_label"].value_counts(dropna=False)


score_label
Counter        53651
Hate Speech    49048
Neutral        32857
Name: count, dtype: int64

In [11]:
religion_cols = [
    'annotator_religion_atheist',
    'annotator_religion_buddhist',
    'annotator_religion_christian',
    'annotator_religion_hindu',
    'annotator_religion_jewish',
    'annotator_religion_mormon',
    'annotator_religion_muslim',
    'annotator_religion_nothing',
    'annotator_religion_other'
]

label_order = ["Hate Speech", "Neutral ", "Counter "]

rows = []

for rel in religion_cols:
    rel_name = rel.replace("annotator_religion_", "")
    subset = df[df[rel] == 1]

    counts = subset["score_label"].value_counts().reindex(label_order, fill_value=0)
    total = counts.sum()
    perc = (counts / total * 100).round(2) if total > 0 else counts

    rows.append({
        "religion": rel_name,
        "total_annotations": int(total),

        "hate": int(counts["Hate Speech"]),
        "h%": float(perc["Hate Speech"]),

        "neutral": int(counts["Neutral "]),
        "n%": float(perc["Neutral "]),

        "counter": int(counts["Counter "]),
        "c%": float(perc["Counter "]),
    })

label_distribution_table = (
    pd.DataFrame(rows)
    .sort_values("total_annotations", ascending=False)
    .reset_index(drop=True)
)

label_distribution_table


,religion,total_annotations,hate,h%,neutral,n%,counter,c%
0,christian,58573,21111,36.04,14368,24.53,23094,39.43
1,nothing,36790,13285,36.11,8814,23.96,14691,39.93
2,atheist,27143,9984,36.78,6453,23.77,10706,39.44
3,other,8511,3098,36.40,2073,24.36,3340,39.24
4,jewish,2171,782,36.02,530,24.41,859,39.57
5,buddhist,2138,768,35.92,500,23.39,870,40.69
6,mormon,1066,386,36.21,258,24.20,422,39.59
7,muslim,969,328,33.85,243,25.08,398,41.07
8,hindu,660,218,33.03,176,26.67,266,40.30


In [12]:
df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")
df["is_hate_speech"] = (df["hate_speech_score"] > 0.5).astype(int)

df["is_hate_speech"].value_counts()

is_hate_speech
0    86508
1    49048
Name: count, dtype: int64

In [ ]:
# just for understanding
# α value	Interpretation
# < 0.20	Poor agreement
# 0.20–0.40	Fair
# 0.40–0.60	Moderate
# 0.60–0.80	Substantial
# > 0.80	Strong


In [13]:
religion_cols = [
    'annotator_religion_atheist',
    'annotator_religion_buddhist',
    'annotator_religion_christian',
    'annotator_religion_hindu',
    'annotator_religion_jewish',
    'annotator_religion_mormon',
    'annotator_religion_muslim',
    'annotator_religion_nothing',
    'annotator_religion_other'
]
rows = []

for rel in religion_cols:
    rel_name = rel.replace("annotator_religion_", "")
    
    subset = df[df[rel] == 1]

    grouped = subset.groupby("comment_id")["is_hate_speech"].apply(list)
    grouped = grouped[grouped.apply(len) >= 2]

    if len(grouped) < 10:
        rows.append({
            "religion": rel_name,
            "comments": len(grouped),
            "krippendorff_alpha": np.nan
        })
        continue

    max_len = grouped.apply(len).max()
    reliability_matrix = [
        labels + [np.nan] * (max_len - len(labels))
        for labels in grouped
    ]

    alpha = krippendorff.alpha(
        reliability_data=reliability_matrix,
        level_of_measurement="nominal"
    )

    rows.append({
        "religion": rel_name,
        "comments": len(grouped),
        "krippendorff_alpha": round(alpha, 3)
    })

krippendorff_table = (
    pd.DataFrame(rows)
    .sort_values("comments", ascending=False)
    .reset_index(drop=True)
)

krippendorff_table


,religion,comments,krippendorff_alpha
0,christian,10433,0.081
1,nothing,4813,0.093
2,atheist,2974,0.084
3,other,348,0.024
4,jewish,103,0.010
5,buddhist,90,-0.019
6,muslim,72,0.001
7,mormon,68,-0.025
8,hindu,53,-0.008


In [28]:

annotator_religion_cols = [
    'annotator_religion_atheist',
    'annotator_religion_buddhist',
    'annotator_religion_christian',
    'annotator_religion_hindu',
    'annotator_religion_jewish',
    'annotator_religion_mormon',
    'annotator_religion_muslim',
    'annotator_religion_nothing',
    'annotator_religion_other'
]


target_religion_cols = [
    'target_religion_christian',
    'target_religion_muslim',
    'target_religion_jewish',
    'target_religion_hindu',
    'target_religion_buddhist',
    'target_religion_atheist',
    'target_religion_mormon',
    'target_religion_other'
]


In [29]:
df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")
df["is_hate_speech"] = df["hate_speech_score"] > 0.5


In [32]:
hate_table = pd.DataFrame(
    index=[c.replace("annotator_religion_", "") for c in annotator_religion_cols],
    columns=[c.replace("target_religion_", "") for c in target_religion_cols],
    dtype=float
)

count_table = hate_table.copy()

for a_col in annotator_religion_cols:
    a_name = a_col.replace("annotator_religion_", "")
    annotator_subset = df[df[a_col] == 1]

    for t_col in target_religion_cols:
        t_name = t_col.replace("target_religion_", "")
        subset = annotator_subset[annotator_subset[t_col] == 1]

        total = len(subset)
        hate_n = int(subset["is_hate_speech"].sum())
        hate_pct = round(hate_n / total * 100, 2) if total > 0 else np.nan

        hate_table.loc[a_name, t_name] = hate_pct
        count_table.loc[a_name, t_name] = hate_n


In [33]:
hate_table


,christian,muslim,jewish,hindu,buddhist,atheist,mormon,other
atheist,10.27,43.23,66.23,22.62,20.00,18.54,15.38,19.82
buddhist,12.95,39.15,62.00,20.00,25.00,5.88,23.08,21.43
christian,13.29,40.79,63.01,23.77,23.80,18.93,19.14,19.26
hindu,3.23,41.82,64.29,12.50,40.00,50.00,14.29,0.00
jewish,13.13,45.45,69.13,22.22,33.33,35.71,28.57,27.03
mormon,18.97,42.31,61.40,27.27,33.33,33.33,20.00,26.32
muslim,12.73,33.02,61.54,31.58,25.00,26.67,18.75,27.27
nothing,11.29,41.54,66.87,21.49,24.34,19.65,19.20,18.81
other,11.06,42.73,64.75,24.36,27.91,24.59,20.69,23.41


In [31]:
count_table


,christian,muslim,jewish,hindu,buddhist,atheist,mormon,other
atheist,136.0,1127.0,951.0,50.0,26.0,33.0,28.0,87.0
buddhist,18.0,83.0,62.0,6.0,4.0,1.0,6.0,9.0
christian,414.0,2138.0,1814.0,140.0,79.0,85.0,80.0,187.0
hindu,1.0,23.0,18.0,1.0,2.0,3.0,1.0,0.0
jewish,13.0,95.0,103.0,4.0,4.0,5.0,4.0,10.0
mormon,11.0,33.0,35.0,3.0,2.0,2.0,3.0,5.0
muslim,7.0,35.0,32.0,6.0,3.0,4.0,3.0,3.0
nothing,214.0,1434.0,1314.0,72.0,46.0,45.0,48.0,127.0
other,46.0,344.0,259.0,19.0,12.0,15.0,12.0,48.0
